In [1]:
from nltk.stem import WordNetLemmatizer
import nltk
import re

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/ruslan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ruslan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/ruslan/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ruslan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
def get_twitter_stopwords():
    with open('data/twitter-stopwords.txt') as f:
        return set(f.readline().split(','))

def get_mpqa_lexicon():
    lexicon = set()
    with open('data/positive_words.txt') as f:
        for word in f.readlines():
            lexicon.add(word.strip())
            lexicon.add(word.strip() + '_neg')
    with open('data/negative_words.txt') as f:
        for word in f.readlines():
            lexicon.add(word.strip())
            lexicon.add(word.strip() + '_neg')
    return lexicon

In [3]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

class TweetTopicPreprocessor:
    def __init__(self):
        self.regex_dict = {
            'EMOJI': u'([\U0001F1E0-\U0001F1FF])|([\U0001F300-\U0001F5FF])|([\U0001F600-\U0001F64F])|([\U0001F680-\U0001F6FF])|([\U0001F700-\U0001F77F])|([\U0001F800-\U0001F8FF])|([\U0001F900-\U0001F9FF])|([\U0001FA00-\U0001FA6F])|([\U0001FA70-\U0001FAFF])|([\U00002702-\U000027B0])|([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])',
            'EMAIL': r"(?:^|(?<=[^\w@.)]))(?:[\w+-](?:\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(?:\.(?:[a-z]{2,})){1,3}(?:$|(?=\b))",
            'CASHTAG': r"(?:[$\u20ac\u00a3\u00a2]\d+(?:[\\.,']\d+)?(?:[MmKkBb](?:n|(?:il(?:lion)?))?)?)|(?:\d+(?:[\\.,']\\d+)?[$\u20ac\u00a3\u00a2])",
            'DATE': r"(?:(?:(?:(?:(?<!:)\b\'?\d{1,4},? ?)?\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b))|(?:(?:(?<!:)\b\\'?\d{1,4},? ?)\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b)?))|(?:\b(?<!\d\\.)(?:(?:(?:[0123]?[0-9][\\.\\-\\/])?[0123]?[0-9][\\.\\-\\/][12][0-9]{3})|(?:[0123]?[0-9][\\.\\-\\/][0123]?[0-9][\\.\\-\\/][12]?[0-9]{2,3}))(?!\.\d)\b))",
            'TIME': r'(?:(?:\d+)?\\.?\d+(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))|(?:(?:[0-2]?[0-9]|[2][0-3]):(?:[0-5][0-9])(?::(?:[0-5][0-9]))?(?: ?(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))?)',
            'EMPHASIS': r"(?:\*\b\w+\b\*)",
            'ELONG': r"\b[A-Za-z]*([a-zA-Z])\1\1[A-Za-z]*\b"
        }

        self.contraction_mapping = {"’": "'", "RT ": " ", "ain't": "is not", "aren't": "are not", "can't": "can not",
                                    "'cause": "because", "could've": "could have",
                                    "couldn't": "could not", "didn't": "did not", "doesn't": "does not",
                                    "don't": "do not", "hadn't": "had not",
                                    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                                    "he's": "he is",
                                    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                                    "how's": "how is", "I'd": "I would",
                                    "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                                    "I've": "I have",
                                    "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                                    "i'll've": "i will have", "i'm": "i am",
                                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                                    "it'll": "it will",
                                    "it'll've": "it will have", "it's": "it is", "it’s": "it is", "let's": "let us",
                                    "ma'am": "madam", "mayn't": "may not",
                                    "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
                                    "must've": "must have",
                                    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                                    "needn't've": "need not have",
                                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                                    "shan't": "shall not",
                                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                                    "she'd've": "she would have",
                                    "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                                    "should've": "should have",
                                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                                    "so's": "so as",
                                    "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                                    "that's": "that is",
                                    "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                                    "here's": "here is",
                                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                                    "they'll've": "they will have",
                                    "they're": "they are", "they've": "they have", "to've": "to have",
                                    "wasn't": "was not", "we'd": "we would",
                                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                                    "we're": "we are", "we've": "we have",
                                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                                    "what're": "what are",
                                    "what's": "what is", "what've": "what have", "when's": "when is",
                                    "when've": "when have", "where'd": "where did",
                                    "where's": "where is", "where've": "where have", "who'll": "who will",
                                    "who'll've": "who will have",
                                    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                                    "will've": "will have",
                                    "won't": "will not", "won't've": "will not have", "would've": "would have",
                                    "wouldn't": "would not",
                                    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                                    "y'all'd've": "you all would have",
                                    "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                                    "you'd've": "you would have",
                                    "you'll": "you will", "you'll've": "you will have", "you're": "you are",
                                    "you've": "you have", "It's": "It is", "You'd": "You would",
                                    ' u ': " you ", 'yrs': 'years', 'FYI': 'For your information', ' im ': ' I am ',
                                    'lol': 'LOL', 'You\'re': 'You are'
            , 'can’t': 'can not', '…': '. ', '...': '. ', '\'\'': '\'', '≠': '', 'ain’t': 'am not', 'I’m': 'I am',
                                    'RT\'s': ''}
        self.emoticons = {
            ':*': '<kiss>',
            ':-*': '<kiss>',
            ':x': '<kiss>',
            ':-)': '<happy>',
            ':-))': '<happy>',
            ':-)))': '<happy>',
            ':-))))': '<happy>',
            ':-)))))': '<happy>',
            ':-))))))': '<happy>',
            ':)': '<happy>',
            ':))': '<happy>',
            ':)))': '<happy>',
            ':))))': '<happy>',
            ':)))))': '<happy>',
            ':))))))': '<happy>',
            ':)))))))': '<happy>',
            ':o)': '<happy>',
            ':]': '<happy>',
            ':3': '<happy>',
            ':c)': '<happy>',
            ':>': '<happy>',
            '=]': '<happy>',
            '8)': '<happy>',
            '=)': '<happy>',
            ':}': '<happy>',
            ':^)': '<happy>',
            '|;-)': '<happy>',
            ":'-)": '<happy>',
            ":')": '<happy>',
            '\o/': '<happy>',
            '*\\0/*': '<happy>',
            ':-D': '<laugh>',
            ':D': '<laugh>',
            '8-D': '<laugh>',
            '8D': '<laugh>',
            'x-D': '<laugh>',
            'xD': '<laugh>',
            'X-D': '<laugh>',
            'XD': '<laugh>',
            '=-D': '<laugh>',
            '=D': '<laugh>',
            '=-3': '<laugh>',
            '=3': '<laugh>',
            'B^D': '<laugh>',
            '>:[': '<sad>',
            ':-(': '<sad>',
            ':-((': '<sad>',
            ':-(((': '<sad>',
            ':-((((': '<sad>',
            ':-(((((': '<sad>',
            ':-((((((': '<sad>',
            ':-(((((((': '<sad>',
            ':(': '<sad>',
            ':((': '<sad>',
            ':(((': '<sad>',
            ':((((': '<sad>',
            ':(((((': '<sad>',
            ':((((((': '<sad>',
            ':(((((((': '<sad>',
            ':((((((((': '<sad>',
            ':-c': '<sad>',
            ':c': '<sad>',
            ':-<': '<sad>',
            ':<': '<sad>',
            ':-[': '<sad>',
            ':[': '<sad>',
            ':{': '<sad>',
            ':-||': '<sad>',
            ':@': '<sad>',
            ":'-(": '<sad>',
            ":'(": '<sad>',
            'D:<': '<sad>',
            'D:': '<sad>',
            'D8': '<sad>',
            'D;': '<sad>',
            'D=': '<sad>',
            'DX': '<sad>',
            'v.v': '<sad>',
            "D-':": '<sad>',
            '(>_<)': '<sad>',
            ':|': '<sad>',
            '>:O': '<surprise>',
            ':-O': '<surprise>',
            ':-o': '<surprise>',
            ':O': '<surprise>',
            '°o°': '<surprise>',
            'o_O': '<surprise>',
            'o_0': '<surprise>',
            'o.O': '<surprise>',
            'o-o': '<surprise>',
            '8-0': '<surprise>',
            '|-O': '<surprise>',
            ';-)': '<wink>',
            ';)': '<wink>',
            '*-)': '<wink>',
            '*)': '<wink>',
            ';-]': '<wink>',
            ';]': '<wink>',
            ';D': '<wink>',
            ';^)': '<wink>',
            ':-,': '<wink>',
            '>:P': '<tong>',
            ':-P': '<tong>',
            ':P': '<tong>',
            'X-P': '<tong>',
            'x-p': '<tong>',
            ':-p': '<tong>',
            ':p': '<tong>',
            '=p': '<tong>',
            ':-Þ': '<tong>',
            ':Þ': '<tong>',
            ':-b': '<tong>',
            ':b': '<tong>',
            ':-&': '<tong>',
            '>:\\': '<annoyed>',
            '>:/': '<annoyed>',
            ':-/': '<annoyed>',
            ':-.': '<annoyed>',
            ':/': '<annoyed>',
            ':\\': '<annoyed>',
            '=/': '<annoyed>',
            '=\\': '<annoyed>',
            ':L': '<annoyed>',
            '=L': '<annoyed>',
            ':S': '<annoyed>',
            '>.<': '<annoyed>',
            ':-|': '<annoyed>',
            '<:-|': '<annoyed>',
            ':-X': '<seallips>',
            ':X': '<seallips>',
            ':-#': '<seallips>',
            ':#': '<seallips>',
            'O:-)': '<angel>',
            '0:-3': '<angel>',
            '0:3': '<angel>',
            '0:-)': '<angel>',
            '0:)': '<angel>',
            '0;^)': '<angel>',
            '>:)': '<devil>',
            '>:D': '<devil>',
            '>:-D': '<devil>',
            '>;)': '<devil>',
            '>:-)': '<devil>',
            '}:-)': '<devil>',
            '}:)': '<devil>',
            '3:-)': '<devil>',
            '3:)': '<devil>',
            'o/\o': '<highfive>',
            '^5': '<highfive>',
            '>_>^': '<highfive>',
            '^<_<': '<highfive>',
            '<3': '<heart>',
            '^3^': '<smile>',
            "(':": '<smile>',
            " > < ": '<smile>',
            "UvU": '<smile>',
            "uwu": '<smile>',
            'UwU': '<smile>'
        }
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.stop=get_twitter_stopwords()

    def get_compiled(self):
        regexes = {k: re.compile(self.regex_dict[k]) for k, v in
                   self.regex_dict.items()}
        return regexes

    def fit(self, tweet):
        regex = self.get_compiled()
        for key, reg in regex.items():
            tweet = regex[key].sub(lambda m: " <" + key + "> ", tweet)
        for word in self.emoticons.keys():
            tweet = tweet.replace(word, self.emoticons[word])
        tweet = tweet.lower()

        # removing stop words
        tweet = ' '.join([word for word in tweet.split() if word not in self.stop])

        for word in self.contraction_mapping.keys():
            tweet = tweet.replace(word, self.contraction_mapping[word])

        pattern_punc = re.compile(r"[\-\"`@#$%^&*(|)/~\[\]{}:;+,.='?!]+")
        tweet = pattern_punc.sub(' ', tweet)

        # lemmatizing
        tweet = ' '.join(map(self.wordnet_lemmatizer.lemmatize, tweet.split()))

        # removing stop words
        tweet = ' '.join([word for word in tweet.split() if word not in self.stop])
        return tweet

In [4]:
class TweetSentimentPreprocessor:
    def __init__(self):
        self.regex_dict = {
            'URL': r"""(?xi)\b(?:(?:https?|ftp|file):\/\/|www\.|ftp\.|pic\.|twitter\.|facebook\.)(?:\([-A-Z0-9+&@#\/%=~_|$?!:;,.]*\)|[-A-Z0-9+&@#\/%=~_|$?!:;,.])*(?:\([-A-Z0-9+&@#\/%=~_|$?!:,.]*\)|[A-Z0-9+&@#\/%=~_|$])""",
            'HASHTAG': r"\#\b[\w\-\_]+\b",
            'MENTION': r"@[A-Za-z0-9]+",
            'EMOJI': u'([\U0001F1E0-\U0001F1FF])|([\U0001F300-\U0001F5FF])|([\U0001F600-\U0001F64F])|([\U0001F680-\U0001F6FF])|([\U0001F700-\U0001F77F])|([\U0001F800-\U0001F8FF])|([\U0001F900-\U0001F9FF])|([\U0001FA00-\U0001FA6F])|([\U0001FA70-\U0001FAFF])|([\U00002702-\U000027B0])|([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])',
            'EMAIL': r"(?:^|(?<=[^\w@.)]))(?:[\w+-](?:\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(?:\.(?:[a-z]{2,})){1,3}(?:$|(?=\b))",
            'CASHTAG': r"(?:[$\u20ac\u00a3\u00a2]\d+(?:[\\.,']\d+)?(?:[MmKkBb](?:n|(?:il(?:lion)?))?)?)|(?:\d+(?:[\\.,']\\d+)?[$\u20ac\u00a3\u00a2])",
            'DATE': r"(?:(?:(?:(?:(?<!:)\b\'?\d{1,4},? ?)?\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b))|(?:(?:(?<!:)\b\\'?\d{1,4},? ?)\b(?:[Jj]an(?:uary)?|[Ff]eb(?:ruary)?|[Mm]ar(?:ch)?|[Aa]pr(?:il)?|May|[Jj]un(?:e)?|[Jj]ul(?:y)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]ct(?:ober)?|[Nn]ov(?:ember)?|[Dd]ec(?:ember)?)\b(?:(?:,? ?\'?)?\d{1,4}(?:st|nd|rd|n?th)?\b(?:[,\\/]? ?\'?\d{2,4}[a-zA-Z]*)?(?: ?- ?\d{2,4}[a-zA-Z]*)?(?!:\d{1,4})\b)?))|(?:\b(?<!\d\\.)(?:(?:(?:[0123]?[0-9][\\.\\-\\/])?[0123]?[0-9][\\.\\-\\/][12][0-9]{3})|(?:[0123]?[0-9][\\.\\-\\/][0123]?[0-9][\\.\\-\\/][12]?[0-9]{2,3}))(?!\.\d)\b))",
            'TIME': r'(?:(?:\d+)?\\.?\d+(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))|(?:(?:[0-2]?[0-9]|[2][0-3]):(?:[0-5][0-9])(?::(?:[0-5][0-9]))?(?: ?(?:AM|PM|am|pm|a\\.m\\.|p\\.m\\.))?)',
            'EMPHASIS': r"(?:\*\b\w+\b\*)",
            'ELONG': r"\b[A-Za-z]*([a-zA-Z])\1\1[A-Za-z]*\b"
        }

        self.contraction_mapping = {"’": "'", "RT ": " ", "ain't": "is not", "aren't": "are not", "can't": "can not",
                                    "'cause": "because", "could've": "could have",
                                    "couldn't": "could not", "didn't": "did not", "doesn't": "does not",
                                    "don't": "do not", "hadn't": "had not",
                                    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
                                    "he's": "he is",
                                    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                                    "how's": "how is", "I'd": "I would",
                                    "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
                                    "I've": "I have",
                                    "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                                    "i'll've": "i will have", "i'm": "i am",
                                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                                    "it'll": "it will",
                                    "it'll've": "it will have", "it's": "it is", "it’s": "it is", "let's": "let us",
                                    "ma'am": "madam", "mayn't": "may not",
                                    "might've": "might have", "mightn't": "might not", "mightn't've": "might not have",
                                    "must've": "must have",
                                    "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                                    "needn't've": "need not have",
                                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
                                    "shan't": "shall not",
                                    "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                                    "she'd've": "she would have",
                                    "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                                    "should've": "should have",
                                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                                    "so's": "so as",
                                    "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                                    "that's": "that is",
                                    "there'd": "there would", "there'd've": "there would have", "there's": "there is",
                                    "here's": "here is",
                                    "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                                    "they'll've": "they will have",
                                    "they're": "they are", "they've": "they have", "to've": "to have",
                                    "wasn't": "was not", "we'd": "we would",
                                    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                                    "we're": "we are", "we've": "we have",
                                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                                    "what're": "what are",
                                    "what's": "what is", "what've": "what have", "when's": "when is",
                                    "when've": "when have", "where'd": "where did",
                                    "where's": "where is", "where've": "where have", "who'll": "who will",
                                    "who'll've": "who will have",
                                    "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have",
                                    "will've": "will have",
                                    "won't": "will not", "won't've": "will not have", "would've": "would have",
                                    "wouldn't": "would not",
                                    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                                    "y'all'd've": "you all would have",
                                    "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                                    "you'd've": "you would have",
                                    "you'll": "you will", "you'll've": "you will have", "you're": "you are",
                                    "you've": "you have", "It's": "It is", "You'd": "You would",
                                    ' u ': " you ", 'yrs': 'years', 'FYI': 'For your information', ' im ': ' I am ',
                                    'lol': 'LOL', 'You\'re': 'You are'
            , 'can’t': 'can not', '…': '. ', '...': '. ', '\'\'': '\'', '≠': '', 'ain’t': 'am not', 'I’m': 'I am',
                                    'RT\'s': ''}
        self.emoticons = {
            ':*': '<kiss>',
            ':-*': '<kiss>',
            ':x': '<kiss>',
            ':-)': '<happy>',
            ':-))': '<happy>',
            ':-)))': '<happy>',
            ':-))))': '<happy>',
            ':-)))))': '<happy>',
            ':-))))))': '<happy>',
            ':)': '<happy>',
            ':))': '<happy>',
            ':)))': '<happy>',
            ':))))': '<happy>',
            ':)))))': '<happy>',
            ':))))))': '<happy>',
            ':)))))))': '<happy>',
            ':o)': '<happy>',
            ':]': '<happy>',
            ':3': '<happy>',
            ':c)': '<happy>',
            ':>': '<happy>',
            '=]': '<happy>',
            '8)': '<happy>',
            '=)': '<happy>',
            ':}': '<happy>',
            ':^)': '<happy>',
            '|;-)': '<happy>',
            ":'-)": '<happy>',
            ":')": '<happy>',
            '\o/': '<happy>',
            '*\\0/*': '<happy>',
            ':-D': '<laugh>',
            ':D': '<laugh>',
            '8-D': '<laugh>',
            '8D': '<laugh>',
            'x-D': '<laugh>',
            'xD': '<laugh>',
            'X-D': '<laugh>',
            'XD': '<laugh>',
            '=-D': '<laugh>',
            '=D': '<laugh>',
            '=-3': '<laugh>',
            '=3': '<laugh>',
            'B^D': '<laugh>',
            '>:[': '<sad>',
            ':-(': '<sad>',
            ':-((': '<sad>',
            ':-(((': '<sad>',
            ':-((((': '<sad>',
            ':-(((((': '<sad>',
            ':-((((((': '<sad>',
            ':-(((((((': '<sad>',
            ':(': '<sad>',
            ':((': '<sad>',
            ':(((': '<sad>',
            ':((((': '<sad>',
            ':(((((': '<sad>',
            ':((((((': '<sad>',
            ':(((((((': '<sad>',
            ':((((((((': '<sad>',
            ':-c': '<sad>',
            ':c': '<sad>',
            ':-<': '<sad>',
            ':<': '<sad>',
            ':-[': '<sad>',
            ':[': '<sad>',
            ':{': '<sad>',
            ':-||': '<sad>',
            ':@': '<sad>',
            ":'-(": '<sad>',
            ":'(": '<sad>',
            'D:<': '<sad>',
            'D:': '<sad>',
            'D8': '<sad>',
            'D;': '<sad>',
            'D=': '<sad>',
            'DX': '<sad>',
            'v.v': '<sad>',
            "D-':": '<sad>',
            '(>_<)': '<sad>',
            ':|': '<sad>',
            '>:O': '<surprise>',
            ':-O': '<surprise>',
            ':-o': '<surprise>',
            ':O': '<surprise>',
            '°o°': '<surprise>',
            'o_O': '<surprise>',
            'o_0': '<surprise>',
            'o.O': '<surprise>',
            'o-o': '<surprise>',
            '8-0': '<surprise>',
            '|-O': '<surprise>',
            ';-)': '<wink>',
            ';)': '<wink>',
            '*-)': '<wink>',
            '*)': '<wink>',
            ';-]': '<wink>',
            ';]': '<wink>',
            ';D': '<wink>',
            ';^)': '<wink>',
            ':-,': '<wink>',
            '>:P': '<tong>',
            ':-P': '<tong>',
            ':P': '<tong>',
            'X-P': '<tong>',
            'x-p': '<tong>',
            ':-p': '<tong>',
            ':p': '<tong>',
            '=p': '<tong>',
            ':-Þ': '<tong>',
            ':Þ': '<tong>',
            ':-b': '<tong>',
            ':b': '<tong>',
            ':-&': '<tong>',
            '>:\\': '<annoyed>',
            '>:/': '<annoyed>',
            ':-/': '<annoyed>',
            ':-.': '<annoyed>',
            ':/': '<annoyed>',
            ':\\': '<annoyed>',
            '=/': '<annoyed>',
            '=\\': '<annoyed>',
            ':L': '<annoyed>',
            '=L': '<annoyed>',
            ':S': '<annoyed>',
            '>.<': '<annoyed>',
            ':-|': '<annoyed>',
            '<:-|': '<annoyed>',
            ':-X': '<seallips>',
            ':X': '<seallips>',
            ':-#': '<seallips>',
            ':#': '<seallips>',
            'O:-)': '<angel>',
            '0:-3': '<angel>',
            '0:3': '<angel>',
            '0:-)': '<angel>',
            '0:)': '<angel>',
            '0;^)': '<angel>',
            '>:)': '<devil>',
            '>:D': '<devil>',
            '>:-D': '<devil>',
            '>;)': '<devil>',
            '>:-)': '<devil>',
            '}:-)': '<devil>',
            '}:)': '<devil>',
            '3:-)': '<devil>',
            '3:)': '<devil>',
            'o/\o': '<highfive>',
            '^5': '<highfive>',
            '>_>^': '<highfive>',
            '^<_<': '<highfive>',
            '<3': '<heart>',
            '^3^': '<smile>',
            "(':": '<smile>',
            " > < ": '<smile>',
            "UvU": '<smile>',
            "uwu": '<smile>',
            'UwU': '<smile>'
        }
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.stop=get_twitter_stopwords()
        self.mpqa_lexicon = get_mpqa_lexicon()

    def get_compiled(self):
        regexes = {k: re.compile(self.regex_dict[k]) for k, v in
                   self.regex_dict.items()}
        return regexes

    def fit(self, tweet):
        regex = self.get_compiled()
        for key, reg in regex.items():
            tweet = regex[key].sub(lambda m: " <" + key + "> ", tweet)
        for word in self.emoticons.keys():
            tweet = tweet.replace(word, self.emoticons[word])
        tweet = tweet.lower()

        for word in self.contraction_mapping.keys():
            tweet = tweet.replace(word, self.contraction_mapping[word])

        pattern_punc = re.compile(r"[()\[\]{}:;,.?!]+")
        add_negation = False
        splitted = tweet.split()
        for i, word in enumerate(splitted):
            if word == 'not' or word == 'no' or word.find('n\'t'):
                add_negation = True
                continue
            if pattern_punc.search(word): # last word
                pos = pattern_punc.search(word).group(0)
                splitted[i] = splitted[i][:pos] + '_neg' + splitted[i][pos:]
                add_negation = False
                continue
            if add_negation:
                splitted[i] = splitted[i] + '_neg'
        tweet = ' '.join(splitted)

        pattern_punc = re.compile(r"[\-\"`@#$%^&*(|)/~\[\]{}:;+,.='?!]+")
        tweet = pattern_punc.sub(' ', tweet)

        # lemmatizing
        tweet = ' '.join(map(self.wordnet_lemmatizer.lemmatize, tweet.split()))

        # get tags
        pos_val = nltk.pos_tag(tweet.split())
        # filter words from dicts
        tweet = ' '.join([word for word, tag in pos_val if self.is_sentiment_word(word, tag)])


        return tweet

    def is_sentiment_word(self, word, tag):
        return self.is_senti_word_net(word, tag) or word in self.mpqa_lexicon

    def penn_to_wn(self, tag):
        """
        Convert between the PennTreebank tags to simple Wordnet tags
        """
        if tag.startswith('J'):
            return wn.ADJ
        elif tag.startswith('N'):
            return wn.NOUN
        elif tag.startswith('R'):
            return wn.ADV
        elif tag.startswith('V'):
            return wn.VERB
        return None

    def is_senti_word_net(self, word, tag):
        wn_tag = self.penn_to_wn(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            return False

        lemma = self.wordnet_lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            return False

        synsets = wn.synsets(word, pos=wn_tag)
        if not synsets:
            return False

        # Take the first sense, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())

        return swn_synset.pos_score() > 0 or swn_synset.neg_score() > 0

In [5]:
from sklearn.feature_extraction import text
from sklearn.pipeline import FeatureUnion,Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC

In [6]:
class TweetTopicClassifier:
    def __init__(self):
        self.encoder=LabelEncoder()
        self.tp = TweetTopicPreprocessor()
        word_vectorizer = text.TfidfVectorizer(
          analyzer='word', ngram_range=(1, 3),
          min_df=2, use_idf=True, sublinear_tf=True)
        char_vectorizer = text.TfidfVectorizer(
          analyzer='char', ngram_range=(3, 5),
          min_df=2, use_idf=True, sublinear_tf=True)
        self.ngrams_vectorizer=Pipeline([('feats', FeatureUnion([('word_ngram', word_vectorizer), ('char_ngram', char_vectorizer),])),])
        self.classifier=LinearSVC()

    def fit(self, X, y):
        encoded_y = self.encoder.fit_transform(y)

        cleaned_X = list(map(self.tp.fit, X))
        vectorized_X = self.ngrams_vectorizer.fit_transform(cleaned_X)
        self.classifier.fit(vectorized_X, encoded_y)

    def predict(self, X):
        cleaned_X = list(map(self.tp.fit, X))
        vectorized_X = self.ngrams_vectorizer.transform(cleaned_X)
        y = self.classifier.predict(vectorized_X)
        return self.encoder.inverse_transform(y)

In [7]:
class TweetSentimentClassifier:
    def __init__(self):
        self.encoder=LabelEncoder()
        self.tp = TweetSentimentPreprocessor()
        word_vectorizer = text.TfidfVectorizer(
          analyzer='word', ngram_range=(1, 3),
          min_df=3, use_idf=True, sublinear_tf=True)
        char_vectorizer = text.TfidfVectorizer(
          analyzer='char', ngram_range=(3, 5),
          min_df=2, use_idf=True, sublinear_tf=True)
        # self.ngrams_vectorizer=Pipeline([('feats', FeatureUnion([('word_ngram', word_vectorizer),])),])
        self.ngrams_vectorizer=Pipeline([('feats', FeatureUnion([('word_ngram', word_vectorizer), ('char_ngram', char_vectorizer),])),])
        self.classifier=LinearSVC()

    def fit(self, X, y):
        encoded_y = self.encoder.fit_transform(y)

        cleaned_X = list(map(self.tp.fit, X))
        vectorized_X = self.ngrams_vectorizer.fit_transform(cleaned_X)
        self.classifier.fit(vectorized_X, encoded_y)

    def predict(self, X):
        cleaned_X = list(map(self.tp.fit, X))
        vectorized_X = self.ngrams_vectorizer.transform(cleaned_X)
        y = self.classifier.predict(vectorized_X)
        return self.encoder.inverse_transform(y)

In [8]:
from pandas import read_csv

train_df = read_csv('data/Train.csv', header=0)
test_df = read_csv('data/Test.csv', header=0)

In [9]:
from sklearn.metrics import classification_report

topic_clf=TweetTopicClassifier()
topic_clf.fit(train_df['TweetText'], train_df['Topic'])
topic_pred=topic_clf.predict(test_df['TweetText'])
print(classification_report(test_df['Topic'],topic_pred))

              precision    recall  f1-score   support

       apple       0.89      0.95      0.92        98
      google       0.94      0.78      0.86        79
   microsoft       0.88      0.78      0.83        78
     twitter       0.73      0.86      0.79        87

    accuracy                           0.85       342
   macro avg       0.86      0.84      0.85       342
weighted avg       0.86      0.85      0.85       342



In [10]:
sentiment_clf=TweetSentimentClassifier()
relevant_train_df = train_df[train_df.Sentiment != 'irrelevant']
relevant_test_df = test_df[test_df.Sentiment != 'irrelevant']
sentiment_clf.fit(relevant_train_df['TweetText'], relevant_train_df['Sentiment'])
sentiment_pred=sentiment_clf.predict(relevant_test_df['TweetText'])
print(classification_report(relevant_test_df['Sentiment'],sentiment_pred))

              precision    recall  f1-score   support

    negative       0.68      0.43      0.53        49
     neutral       0.76      0.90      0.82       156
    positive       0.55      0.38      0.44        32

    accuracy                           0.73       237
   macro avg       0.66      0.57      0.60       237
weighted avg       0.71      0.73      0.71       237



In [13]:
example_tweet = 'I am not using iphones, it is too expensive'
print(topic_clf.predict([example_tweet])[0])
print(sentiment_clf.predict([example_tweet])[0])

apple
negative
